In [10]:
import argparse
from pathlib import Path
import sys
from datetime import datetime

wd = Path('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/workflows/BB-NF_17m')
sys.path.append('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/python')
from eddyproconfigeditor import EddyproConfigEditor

In [11]:
arcc_wd = Path('/project/eddycovworkflow/afox18/Platinum-EddyPro7/workflow/BB-NF_17m')
data_dir = Path('/gscratch/afox18/eddycovworkflow/InputData/Chimney')
out_dir = Path('/gscratch/afox18/eddycovworkflow/ExpectedOutputs/Chimney/BB-NF/17m')

In [12]:
template = EddyproConfigEditor(wd / 'ini/BB-NF_17m_template.eddypro')

In [13]:
template.Proj.set_metadata(arcc_wd / 'ini/BB-NF_17m.metadata')
template.Proj.set_biomet(mode='dir', path=data_dir / 'biomet/BB-NF/EC/17m/EddyPro_Biomet', extension='dat', subfolders=False)
template.Proj.set_project_name('BB-NF_17m')

In [14]:
template.Basic.set_raw_data(path=data_dir / 'raw/BB-NF/EC/17m/Calibrated', fmt='yyyy_mm_dd_HHMM.dat', subfolders=False)
template.Basic.set_out_path(out_dir / 'Template')
template.Basic.set_project_date_range(start='2019-01-01 00:00', end='2024-12-31 23:30')
template.Basic.set_missing_samples_allowance(pct=10)
template.Basic.set_flux_averaging_interval(minutes=30)
template.Basic.set_north_reference(method='mag')
template.Basic.set_output_id(output_id='template')

In [15]:
template.Adv.Proc.set_wind_speed_measurement_offsets(0, 0, 0)
template.Adv.Proc.set_axis_rotations_for_tilt_correction(
    method='planar_fit',
    configure_planar_fit_settings_kwargs=dict(
        w_max=0.5,
        u_min=0.5,
        num_per_sector_min=30,
        start='all_available',
        end='all_available',
        fix_method='CW',
        north_offset=0,
        sectors=[(False, 90)]*4
    )
)
template.Adv.Proc.set_turbulent_fluctuations(detrend_method='block')
template.Adv.Proc.set_timelag_compensations(method='covariance_maximization_with_default')
template.Adv.Proc.set_compensation_of_density_fluctuations(
    burba_method='multiple',
    day_bot=[3.0935, -0.0819, 0.0018, -0.3601],
    day_top=[0.5773, -0.0107, 0.0012, -0.0914],
    day_spar=[0.7714, -0.0154, 0.0011, -0.1164],
    night_bot=[2.2022, -0.122, 0, -0.3001],
    night_top=[-0.2505, -0.0303, 0, 0.0556],
    night_spar=[0.0219, -0.0361, 0, 0.0145]
)

In [16]:
template.Adv.Out.set_spectral_output(
    binned_spectra=True,
    binned_ogives=True,
    ensemble_spectra=True,
    full_spectra='none',
    full_cospectra='none'
)
template.Adv.Out.set_chain_of_custody(
    unprocessed='stats',
    despiked='stats',
    timelag_corrected='timeseries',
    variables=['u', 'v', 'w', 'ts', 'co2', 'h2o', ]
)

In [17]:
template.Basic.get_out_path()

PosixPath('/gscratch/afox18/eddycovworkflow/ExpectedOutputs/Chimney/BB-NF/17m/Template')

In [18]:
template.to_eddypro(ini_file=wd / 'ini/BB-NF_17m_template.eddypro')
template.to_eddypro_parallel(
    ini_dir=wd / 'ini/BB-NF_17m_template_parallel',
    out_path=out_dir / 'Template_Parallel',
    file_duration=1440,
    worker_windows=[datetime(y, 1, 1, 0, 0, 0) for y in range(2019, 2025)]
)

In [19]:
template.Adv.Spec.get_calculation()

{'start': 'project',
 'end': 'project',
 'window': 'hamming',
 'bins': 50,
 'power_2': True}

In [22]:
template.Adv.Proc._configure_planar_fit_settings(return_inputs=True)

TypeError: EddyproConfigEditor._Adv._Proc._configure_planar_fit_settings() missing 3 required positional arguments: 'w_max', 'u_min', and 'num_per_sector_min'

In [21]:
template.Adv.Proc.get_axis_rotations_for_tilt_correction()

{'method': 'planar_fit',
 'pf_file': None,
 'configure_planar_fit_settings_kwargs': {'pf_subset': 0,
  'u_min': 0.5,
  'w_max': 0.5,
  'num_per_sector_min': 30,
  'fix_method': 'CW',
  'north_offset': 0.0,
  'sectors': [(0, 90.0), (0, 90.0), (0, 90.0), (0, 90.0)]}}